<a href="https://colab.research.google.com/github/thoov3r98/DataQualityHPPredictions/blob/main/HPPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd
import sklearn as sk
import numpy as np
from datascience import *
import os
from pathlib import Path
import nltk as nl
import statistics
from datetime import date 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from flaml.default import preprocess_and_suggest_hyperparams
from flaml import AutoML

/usr/local/lib/python3.7/dist-packages/datascience/tables.py:17: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)
/usr/local/lib/python3.7/dist-packages/datascience/util.py:10: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)


In [ ]:
pip install flaml

     |████████████████████████████████| 157 kB 15.2 MB/s 
     |████████████████████████████████| 2.0 MB 35.5 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [ ]:
import flaml

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class BestHyperparamFinder:
  def __init__(self, dataName):
    self.dataname = dataName 
    self.path = "/content/drive/Shared drives/Capstone 440/Data/" + dataName + ".csv"


  def findHyperparams(self, givenTarget = False, givenTask = False):
    np.random.seed(100)
    data = pd.read_csv(self.path)
    for i in list(data):
      if "Unnamed: 0" == i:
        data=data.drop(columns="Unnamed: 0")
      break
    if not givenTarget:
      answer = input("Please enter target variable: ")
    else:
      answer = givenTarget
    y = pd.DataFrame()
    x = data
    y[answer] = data[answer]
    x = x.drop(columns = [answer])
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    y_train = np.array(y_train)
    if not givenTask:
      task = input("Please enter type of prediction (regression or classification):  ")
    else:
        task = givenTask
    automl = AutoML()
    automl_settings = {"time_budget": 300}
    automl.fit(X_train, y_train, task=task, estimator_list=["xgboost"], verbose = 0)
    ans = automl.predict(X_test)
    if task == 'regression':
      print('For Regression Task')
      print(automl.best_config)
      print('Mean Squared Error:', mean_squared_error(y_test, ans))
      return automl.best_config
    else:
      print('For Classification Task')
      print(automl.best_config)
      print('Accuracy:', accuracy_score(y_test, ans))
      return automl.best_config

In [ ]:
metaDataDictionary = pd.read_csv("/content/drive/Shared drives/Capstone 440/Data/" + "Meta Data3.csv")



In [ ]:
metaDataDictionary

,Date Created,Dataset,Target Variable,Task,link,Checked
0,1988-07-01,iris,variety,classification,NaN,Good
1,1995-12-01,abalone,Class_number_of_rings,regression,NaN,Good
2,27 April 1987,mushrooms,class,classification,NaN,Good
3,unknown,drug200,Drug,classification,NaN,good
4,unknown,kag_risk_factors_cervical_cancer,Dx:Cancer,classification,https://www.kaggle.com/datasets/loveall/cervic...,good
...,...,...,...,...,...,...
71,NaN,SynchronousMachine,I_f,regression,NaN,NaN
72,NaN,company,Sales,regression,NaN,NaN
73,NaN,GoldUP,Gold_Price,regression,NaN,NaN
74,NaN,Cricket_chirps,Y,regression,NaN,NaN


In [ ]:

targetDict = {}

for i in range(len(metaDataDictionary['Dataset'])):
  print(i)
  print(metaDataDictionary['Dataset'][i])
  HPFinder = BestHyperparamFinder(metaDataDictionary['Dataset'][i])
  targetHP = HPFinder.findHyperparams(givenTarget = metaDataDictionary['Target Variable'][i], givenTask = metaDataDictionary['Task'][i])
  targetHP['Data Name'] = HPFinder.dataname
  targetDict[HPFinder.dataname] = targetHP
    
    



0
iris
For Classification Task
{'n_estimators': 14, 'max_leaves': 7, 'min_child_weight': 0.001, 'learning_rate': 1.0, 'subsample': 0.1119632717423609, 'colsample_bylevel': 0.616863540718967, 'colsample_bytree': 0.6262289096281859, 'reg_alpha': 0.0023841774773573966, 'reg_lambda': 0.6672753885620387}
Accuracy: 1.0
1
abalone
For Regression Task
{'n_estimators': 40, 'max_leaves': 18, 'min_child_weight': 35.210577564862284, 'learning_rate': 0.1153208607212986, 'subsample': 1.0, 'colsample_bylevel': 0.8714379245380952, 'colsample_bytree': 0.9861072741057454, 'reg_alpha': 0.0009765625, 'reg_lambda': 0.3181866149088085}
Mean Squared Error: 4.914496445453599
2
mushrooms
For Classification Task
{'n_estimators': 28, 'max_leaves': 4, 'min_child_weight': 2.0028527505663547, 'learning_rate': 1.0, 'subsample': 0.8013146656246004, 'colsample_bylevel': 0.8298115767292761, 'colsample_bytree': 0.938486490665451, 'reg_alpha': 0.001077418800856355, 'reg_lambda': 0.3030435039505446}
Accuracy: 1.0
3
drug200

In [ ]:
targetDict

{'CASP': {'Data Name': 'CASP',
  'colsample_bylevel': 0.6839382582087216,
  'colsample_bytree': 0.6059459641267175,
  'learning_rate': 0.11598916644682901,
  'max_leaves': 277,
  'min_child_weight': 44.52425611856729,
  'n_estimators': 211,
  'reg_alpha': 0.02184723146387385,
  'reg_lambda': 1.1734228267878368,
  'subsample': 0.8413048297641477},
 'Concrete_Data': {'Data Name': 'Concrete_Data',
  'colsample_bylevel': 0.8672138416214238,
  'colsample_bytree': 0.6101631329427205,
  'learning_rate': 0.07405282594092845,
  'max_leaves': 7,
  'min_child_weight': 8.22173778686909,
  'n_estimators': 555,
  'reg_alpha': 0.12760289881653272,
  'reg_lambda': 0.03894982398392355,
  'subsample': 1.0},
 'Cricket_chirps': {'Data Name': 'Cricket_chirps',
  'colsample_bylevel': 1.0,
  'colsample_bytree': 1.0,
  'learning_rate': 1.0,
  'max_leaves': 4,
  'min_child_weight': 65.24118435167922,
  'n_estimators': 5,
  'reg_alpha': 0.002236475433401577,
  'reg_lambda': 0.49132577944409994,
  'subsample': 0

In [ ]:
HPTargetDF = pd.DataFrame.from_dict(targetDict, orient ='index')

In [ ]:
HPTargetDF

,n_estimators,max_leaves,min_child_weight,learning_rate,subsample,colsample_bylevel,colsample_bytree,reg_alpha,reg_lambda,Data Name,FLAML_sample_size
iris,14,7,0.001000,1.000000,0.111963,0.616864,0.626229,0.002384,0.667275,iris,NaN
abalone,40,18,35.210578,0.115321,1.000000,0.871438,0.986107,0.000977,0.318187,abalone,NaN
mushrooms,28,4,2.002853,1.000000,0.801315,0.829812,0.938486,0.001077,0.303044,mushrooms,NaN
drug200,18,8,0.038430,1.000000,1.000000,0.963462,1.000000,0.000977,0.022391,drug200,NaN
kag_risk_factors_cervical_cancer,13,6,0.267793,0.303591,1.000000,0.788991,0.814256,0.000977,1.238141,kag_risk_factors_cervical_cancer,NaN
...,...,...,...,...,...,...,...,...,...,...,...
SynchronousMachine,434,6,0.170683,0.029988,1.000000,1.000000,1.000000,0.000977,0.084444,SynchronousMachine,NaN
company,159,14,2.168947,0.088548,0.638395,0.711557,1.000000,0.002321,0.000977,company,NaN
GoldUP,92,74,0.001154,0.173940,0.565121,0.898595,0.936817,0.000977,0.651345,GoldUP,NaN
Cricket_chirps,5,4,65.241184,1.000000,0.848078,1.000000,1.000000,0.002236,0.491326,Cricket_chirps,NaN


In [ ]:
HPTargetDF.to_csv('HPTargetDF.csv')

In [ ]:
files.download('HPTargetDF.csv')